![](image/icon1.png) ![](image/icon2.png) ![](image/icon3.png) 

    
>**Projet Informatique L2** **UT3 Paul Sabatier**  
>**Paul Chiapello**   
>**Nathan DELPORTE**

## Projet RayTracing
***

## Objectif:
***
L'objectif de ce projet consiste à créer une image à partir de lancer de rayons.
Pour cela il a fallu créer plusieurs élements telles que la scène en y intégrant des objets tels que des sphères et plans faisant office de surface et permettant une impression de profondeur.
Pour finir on a essayé de créer un petit film mettant en scène une balle qui rebondi à partir d’un enchaînement de création d’image.
***
Avant de commencer le projet, nous avons procédé à l'installation de nos interpréteurs Python et de nos terminaux à l'aide de GitBash. Nous avons ensuite créé un dépôt sur GitHub pour travailler en collaboration, voici le lien : [Projet_Info_Image](https://github.com/Paulciapello/Projet_Info_Image.git)

Nous avons décidé d'utiliser l'interpréteur Jupyter en raison de sa simplicité et de la possibilité de créer un rapport directement dans l'environnement.

## Déroulé du projet: 
***

1) Création des variables des objets sur la scène
2) Création des normales et des rayons
3) Calcul des intersections avec les objets
4) Mise en places des éclairages, des ombres et des réflections
5) Paramètre renseignés pour l’image
6) Mise en place de l'animation
7) Projet Initial (Echouée)
***

### I.	Création des dictionnaires des objets sur la scène
***
Chaque objet nous les avons rangé dans des fonctions qui faisaient en quelque sorte office de répertoire pour leur variables respectives.
En effet leur variables respective qui se définissent sous forme de liste sont les propriétés de l'objet. On peut voir que chaque propriété est identifiée grâce à une balise. Et chaque balise se définit grâce au variable appelé lors de la création de la fonction avec des “np.array”.


#### Rayon

In [2]:
def create_Ray(O, D):
    ray = {'origin': np.array(O),
           'direction': np.array(D)}
    return ray

#### Plan

In [3]:
def create_plane(P, n, amb, dif, sp,ref, i):
    plane = {'type' : 'plane',
             'position': np.array(P),
             'vect_n': np.array(n),
             'diffuse': np.array(dif),
             'ambient' : np.array(amb),
             'specular': np.array(sp),
             'reflection': ref,
             'index' : int(i)}
    return plane

#### Sphere

In [5]:
def create_sphere(A,lim,C, r, amb, dif, sp, ref, i):
    sphere = {'type':'sphere',                                                                  
              'axe':np.array(A),
              'height':np.array(lim),
              'centre': np.array(C),
              'rayon': np.array(r),
              'diffuse': np.array(dif),
              'ambient' : np.array(amb),
              'specular': np.array(sp),
              'reflection': ref,
              'index' : int(i)}
    return sphere

### II.	Création des normales et des rayons
***

#### Fonction de Normalisation

In [ ]:
def normalize(x):
    return x/np.linalg.norm(x)

#### Rayon definie pour tous t

In [ ]:
def rayAt(ray, t):
    return ray['origin']+t*ray['direction']

#### Fonction de normal aux surfaces 

In [ ]:
def get_Normal(obj, M):
    if obj['type'] == 'sphere':
        Vect =normalize(M-obj['centre'])
    elif obj['type'] == 'plane':
        Vect = obj['vect_n']
    elif obj['type'] == 'cylinder':
        H = obj['centre']
        Vect = normalize(M[0]-H[0],M[1]-H[1],0)
    
    return Vect

### IV. Calculs des intersections avec les objets


***

#### Fonction d'intersection rayon/plan

In [ ]:
def intersect_Plane(ray, plane):
    P = plane['position']
    n = plane['vect_n']
    O = ray['origin']
    d = ray['direction']
    if abs(np.dot(d, n)) < 1e-6:
        return np.inf
    t = -np.dot((O-P),n)/np.dot(d, n)
    if t>0:
        return t
    return np.inf

#### Fonction d'intersection rayon/sphere

In [1]:
def intersect_Sphere(ray, sphere):
    O = ray['origin']
    d = ray['direction']
    C = sphere['centre']
    r = sphere['rayon']

    a = np.dot(d, d)
    b = -2 * np.dot(d, (C-O))
    c = np.dot((C-O),(C-O)) - r**2

    delta = b**2 - 4*a*c
    if delta >= 0:
        t1 = (-b-np.sqrt(delta))/(2*a)
        t2 = (-b+np.sqrt(delta))/(2*a)
        if t1 >= 0 and t2 >= 0:
            return min(t1, t2)
        else:
            return np.inf
    else:
        return np.inf

#### Fonction d'intersection rayon/cylindre

In [2]:
def intersect_Cylinder(ray, cylinder):
    O = ray['origin']
    d = ray['direction']
    C = cylinder['centre']
    R = cylinder['rayon']
    z = 10
    
    x1 = O[0]
    y1 = O[1]       #récupere les (x1,y1,z1) de O
    #O_z = O[2]
    
    a = d[0]
    b = d[1]       #récupere les (a,b,c) du vecteur direction
    #c = d[2]
    
    x0 = C[0]
    y0 = C[1]       #récupere les(xo,yo,zo) de C
    z0 = C[2]
    
    A = a**2 + b**2
    B = 2 * (x1 * a - x0 * a - y1 * b + y0 * b)
    C = x1**2 - 2 * x1 * x0 + x0**2 + y1**2 - 2 * y1 * y0 + y0**2 - R**2 + (z - z0)**2
    
    delta = B**2 - 4 * A * C
    
    if delta >= 0:
        t_1 = (-B-np.sqrt(delta))/(2*A)
        t_2 = (-B+np.sqrt(delta))/(2*A)
        if t_1 >= 0 and t_2 >= 0:
            return min(t_1, t_2)
        else:
            return np.inf
    else:
        return np.inf
    
def intersect_Scene(ray, obj):
    if obj['type'] == 'plane':
        return intersect_Plane(ray, obj)
    elif obj['type'] == 'sphere':
        return intersect_Sphere(ray, obj)
    elif obj['type'] == 'cylinder':
        return intersect_Cylinder(ray, obj)

### V. Mise en places des eclairages, des ombres et des réflections

***

L'objectif de ce code est de voir si le point "P" qui est sur le plan à une normale "N" qui a une intersection avec un objet vers la source lumineuse. Si l'objet est dans l'ombre cela nous renverra un "true" sinon un false. Ce booléen sera pris en compte dans la fonction "trace_ray".

In [3]:
def Is_in_Shadow(obj_min,P,N):
    PL = normalize(L-P)
    rayTest = create_Ray(P+acne_eps*N,PL)
    I_intersect = []
    for obj in scene :
        if obj['index'] != obj_min['index']:
            t_obj = intersect_Scene(rayTest, obj)
            if t_obj != np.inf:
                I_intersect.append(t_obj)
    if I_intersect : 
        return True
    return False

Puis l’eclairage, on a récupéré les normal entre Le plan et la lumière mais aussi le plan et la sphère dans des variable respective (PL et PC) mais aussi la normal des objets grâce à la variable N qui prend la valeur du get_Normal. 
A partir de là, il y a “ambient” pour la couleur de l’objet. 
Le “diffuse”  va émettre la lumière, couleur dans plusieurs directions.
Le “specular” qui lui va être en quelque sorte le rond blanc serait la surface dont la normale est la plus proche de la source lumineuse.


In [5]:
def eclairage(obj,Light,P): 
    N = get_Normal(obj, P)
    PL = normalize(L-P)
    PC = normalize(C-P)
#on calcule la couleur suivant les modèles utilisé
    ct = obj['ambient']*Light['ambient']
#Lambert shading (diffuse).
    ct += obj['diffuse']*Light['diffuse']*max(np.dot(N,PL),0)
#Blinn-Phong shading (specular).
    ct += obj['specular']*Light['specular']*max(np.dot(N,normalize(PL+PC)),0)**(materialShininess)
    return ct